In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('E:\DataSets\CovidChestXRay\Chest_xray_Corona_Metadata.csv')

In [3]:
df.head()

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [4]:
print(set(df['Label']))

{'Normal', 'Pnemonia'}


In [5]:
print(df.columns)


Index(['Unnamed: 0', 'X_ray_image_name', 'Label', 'Dataset_type',
       'Label_2_Virus_category', 'Label_1_Virus_category'],
      dtype='object')


In [6]:
for column in df.columns[2:]:
    df[column] = LabelEncoder().fit_transform(df[column])

In [7]:
output_columns = np.array([df.columns[2], df.columns[4], df.columns[5]])
print(output_columns)

['Label' 'Label_2_Virus_category' 'Label_1_Virus_category']


In [8]:
df.head()

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,0,1,4,3
1,1,IM-0127-0001.jpeg,0,1,4,3
2,2,IM-0125-0001.jpeg,0,1,4,3
3,3,IM-0122-0001.jpeg,0,1,4,3
4,4,IM-0119-0001.jpeg,0,1,4,3


In [9]:
total_encoded = []
for column in output_columns:
    col_values = df[column]
    rows = np.arange(col_values.size)
    encoded = np.zeros((col_values.size, col_values.max() + 1))
    encoded[rows, col_values] = 1
    total_encoded.append(encoded)
print(total_encoded)

[array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]]), array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]]), array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])]


In [10]:
stacked = np.hstack((total_encoded[0], total_encoded[1]))
stacked = np.hstack((stacked, total_encoded[2]))

In [11]:
print(stacked.shape)

(5910, 11)


In [12]:
new_df = pd.DataFrame(stacked)
new_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [13]:
new_df['X_ray_image_name'] = df['X_ray_image_name']

In [14]:
new_df.head()

,0,1,2,3,4,5,6,7,8,9,10,X_ray_image_name
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,IM-0128-0001.jpeg
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,IM-0127-0001.jpeg
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,IM-0125-0001.jpeg
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,IM-0122-0001.jpeg
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,IM-0119-0001.jpeg


In [15]:
df = df.astype({'Label': np.float32, 'Label_2_Virus_category': np.float32, 'Label_1_Virus_category': np.float32 })
df.head()

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,0.0,1,4.0,3.0
1,1,IM-0127-0001.jpeg,0.0,1,4.0,3.0
2,2,IM-0125-0001.jpeg,0.0,1,4.0,3.0
3,3,IM-0122-0001.jpeg,0.0,1,4.0,3.0
4,4,IM-0119-0001.jpeg,0.0,1,4.0,3.0


In [16]:
train_df = pd.DataFrame(columns = new_df.columns)
test_df = pd.DataFrame(columns = new_df.columns)

for i in range(len(new_df)):
        if df.iloc[i]['Dataset_type'] == 1:
            train_df = train_df.append(dict(new_df.iloc[i]), ignore_index = True)
        else:
            test_df = test_df.append(dict(new_df.iloc[i]), ignore_index = True)

In [17]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,X_ray_image_name
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,IM-0128-0001.jpeg
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,IM-0127-0001.jpeg
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,IM-0125-0001.jpeg
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,IM-0122-0001.jpeg
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,IM-0119-0001.jpeg


In [18]:
print(train_df.shape)

(5286, 12)


In [19]:
df = pd.read_csv('E:\DataSets\CovidChestXRay\Chest_xray_Corona_Metadata.csv')

In [20]:
df_train = df[df['Dataset_type'] == 'TRAIN']
df_test = df[df['Dataset_type'] == 'TEST']

In [21]:
df_train.head()

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [22]:
train_df.columns[:2]

Index([0, 1], dtype='object')

In [23]:
batch_size = 32
target_size = (1024, 1024)

In [49]:
train_gen = ImageDataGenerator(validation_split = 0.2, rescale=1. / 255)
train_generator = train_gen.flow_from_dataframe(
    train_df,
    directory = r'E:\DataSets\CovidChestXRay\Coronahack-Chest-XRay-Dataset\train',
    x_col = 'X_ray_image_name',
    y_col = train_df.columns[:2],
    target_size = target_size,
    class_mode = 'raw',
    subset = 'training',
    color_mode='grayscale',
    batch_size = batch_size,
    shuffle = True
)
validation_generator = train_gen.flow_from_dataframe(
    train_df,
    directory = r'E:\DataSets\CovidChestXRay\Coronahack-Chest-XRay-Dataset\train',
    x_col = 'X_ray_image_name',
    y_col = train_df.columns[:2],
    target_size = target_size,
    class_mode = 'raw',
    subset = 'validation',
    color_mode='grayscale',
    batch_size = batch_size,
    shuffle = True
)
test_generator = ImageDataGenerator(rescale=1. / 255).flow_from_dataframe(
    test_df,
    directory = r'E:\DataSets\CovidChestXRay\Coronahack-Chest-XRay-Dataset\test',
    x_col = 'X_ray_image_name',
    y_col = train_df.columns[:2],
    target_size = target_size,
    class_mode = 'raw',
    color_mode='grayscale',
    batch_size = batch_size,
    shuffle = False
)

Found 4229 validated image filenames.
Found 1057 validated image filenames.
Found 624 validated image filenames.


In [50]:
print(target_size + (3,))
print(target_size)

(1024, 1024, 3)
(1024, 1024)


In [51]:
from tensorflow.keras.applications.efficientnet import EfficientNetB1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras import layers

In [52]:
eff = EfficientNetB1(include_top = False, input_shape = target_size + (3,))
for layer in eff.layers:
    layer.trainable = False

In [53]:
model = Sequential([
    layers.Conv2D(2, (11, 11), input_shape = target_size + (1,), padding = 'valid'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.5),
    layers.Conv2D(4, (9, 9), padding = 'valid'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.5),
    layers.Conv2D(8, (7, 7), padding = 'valid'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.5),
    layers.Conv2D(16, (5, 5), padding = 'valid'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.5),
    layers.Conv2D(32, (3, 3), padding = 'valid'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(axis=-1),
    layers.Dropout(0.5),
    layers.Conv2D(64, (1, 1), padding = 'valid'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(axis=-1),
    layers.Flatten(),
    layers.Dense(512, activation = 'relu'),
    layers.Dropout(0.8),
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(32, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation = 'sigmoid')
])
model.summary()
'''model = Sequential()
model.add(eff)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation = 'softmax'))
model.summary()'''

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 1014, 1014, 2)     244       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 507, 507, 2)       0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 507, 507, 2)       8         
_________________________________________________________________
dropout_24 (Dropout)         (None, 507, 507, 2)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 499, 499, 4)       652       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 249, 249, 4)       0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 249, 249, 4)      

"model = Sequential()\nmodel.add(eff)\nmodel.add(GlobalAveragePooling2D())\nmodel.add(Flatten())\nmodel.add(Dense(1024, activation = 'relu'))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(512, activation = 'relu'))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(64, activation = 'relu'))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(2, activation = 'softmax'))\nmodel.summary()"

In [54]:
from tensorflow.keras import callbacks
plateau = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                     patience = 1,
                                     factor = 0.1,
                                     min_lr = 0.0001,
                                     mode = 'max')

early_stopping = callbacks.EarlyStopping(monitor = 'val_loss',
                                        patience = 3,
                                        restore_best_weights=True,
                                        mode = 'max')

In [55]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [56]:
epochs = 10

In [ ]:
ssTrain = train_generator.n//batch_size
ssValidation = validation_generator.n//batch_size

history = model.fit(
    train_generator,
    epochs = epochs,
    steps_per_epoch = ssTrain,
    validation_data = validation_generator,
    validation_steps = ssValidation
)

Epoch 1/10
 16/132 [==>...........................] - ETA: 33s - loss: 1.3683 - accuracy: 0.8164

In [38]:
model.evaluate(test_generator)

20/20 [==============================] - 11s 530ms/step - loss: 0.8412 - accuracy: 0.6250


[0.841163158416748, 0.625]

In [39]:
model.predict(test_generator)

array([[0.8492143 ],
       [0.86813045],
       [0.85931474],
       [0.89934605],
       [0.8836457 ],
       [0.8845061 ],
       [0.8890163 ],
       [0.8799163 ],
       [0.8590146 ],
       [0.8900675 ],
       [0.85487604],
       [0.85679644],
       [0.88083756],
       [0.8774391 ],
       [0.85939264],
       [0.87018216],
       [0.9140719 ],
       [0.8720351 ],
       [0.9116428 ],
       [0.87082624],
       [0.8754712 ],
       [0.887974  ],
       [0.8773    ],
       [0.92750525],
       [0.92068774],
       [0.9685759 ],
       [0.9164483 ],
       [0.8482693 ],
       [0.86666805],
       [0.85632277],
       [0.80010295],
       [0.85341895],
       [0.89829546],
       [0.87743413],
       [0.8630934 ],
       [0.85747445],
       [0.8706935 ],
       [0.873853  ],
       [0.90644175],
       [0.94911397],
       [0.8907295 ],
       [0.92107344],
       [0.82191265],
       [0.85823977],
       [0.86937076],
       [0.8678388 ],
       [0.9375642 ],
       [0.874